# Sreality - Vizualizace
## Jirka Zelenka
### 12.3.-24.4.2020
### Celý projekt = Scraping + Cleaning & Dropping + Vizualizace + All_In_One + PowerBI
-----------------------------------------------------------------------------

### Obsah:
* I) Importování Packagů
* II) Vizualizace


-----------------------
# I) Importování Packagů
-----------------------

-----------------------
# II) Vizualiazce
* https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
-----------------------


#### 0) Nadefinované grafické pomůcky
----
#### 1) Počty inzerátů
 * 1.a) Podle místností
 * 1.b) Podle měst/okresu/kraje
---
#### 2) Ceny
- 2.a) Podle místností
- 2.b) Podle měst
- - 2.b.1) Nejdražší města
- - 2.b.2) Nejlevnější města
- 2.c) Podle okresů
- - 2.c.1) Nejdražší okresy
- - 2.c.2) Nejlevnější okresy
- 2.d) Podle krajů
-----------------------
#### 3) Ceny podle místností a oblastí PRAHA & MOST
- 3.a) Ceny podle místností Praha
- 3.b) Ceny městských částí Prahy
- - 3.b.1) Nejdražší městské části
- - 3.b.2) Nejlevnější pražské části
- 3.c) Ceny podle místností Most
- 3.d) PRAHA vs MOST
-----------------------
#### 4) Pivot - Pražské části a pokoje
#### 4.2) Pivot - Kraje a Pokoje - poty inzerátů // Ceny // Ceny_metr
-----------------------
#### 5) Analýzy podle okresů a krajů = DODĚLAT
----
#### 6) Interaktivní widgety = DODĚLAT
---

### 0) Nadefinované grafické pomůcky

In [ ]:
### Barvičky do BARů

# červená a zelená jsou vidět na html exportu, modrá a rudá s černým písmem NE
# Takže na HTML export musím místo modré dát tyrkys...

zelená = "#22c02c"
tyrkys =   "#16c9ea"
modrá  =    "#0307ca"
červená =  "#e61717"
rudá =      "#a10000"
hnědá = "#4d0000"


# 1.) Zkusi barevnou škálu na celou tabulku. tím by se našly extrémy co nejdou podle tmavosti
# 2) Pivot - udělat jednak potlačení nul, a pak barevnou paletu?
# 3) Zkusit nějak aplikovat ten sloupec s dvěma arvami, ale musím mít kladné i záporné hodnoty, nějaké odchylky od průměru - to bych asi msuel třeba na všechny okresy najednou nebo něco


########################################################################

def small_red(val):
 
    color = 'red' if val < 500 else 'white'
    return 'color: %s' % color

 #.style.applymap(small_red)
########################################################################

def highlight_max(s):
 
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

#  .style.apply(highlight_max) 

########################################################################
def highlight_median(s):   # Vyznačí nejvyšší hodnotu ve sloupci

    is_max = s == s.median()
    return ['background-color: green' if v else '' for v in is_max]

#  .style.apply(highlight_median) 


########################################################################
########################################################################

# https://jakevdp.github.io/PythonDataScienceHandbook/04.07-customizing-colorbars.html



### 1) Počty inzerátů

### 1.a) Počty inzerátů podle místností

In [ ]:

#data.pokoje.value_counts()
#data.pokoje.value_counts().to_frame()

data.pokoje.value_counts().to_frame().style.bar(align = "mid", color = tyrkys)   # Mid je zásadní, aby barevné slooupce měy správné proporce

#ALternativně:
#data.groupby("pokoje")['url_id'].count().nlargest(10).to_frame().style.bar(align = "mid", color = tyrkys)

### 1.b) Počty inzerátů podle měst/okresu/kraje

In [ ]:
########################################################
# Třikrát identické funkce: ?? 

#data.groupby()[].nunique()  // .count() //.size()


In [ ]:
# 10 Měst s nejvíce inzeráty
data.groupby(['město', "okres"])['url_id'].count().nlargest(10).to_frame().style.bar(align = "mid", color = tyrkys)

In [ ]:
# 10 Měst s nejvíce inzeráty bez Prahy
data[data["město"] != "Praha"].groupby(['město', "okres"])['url_id'].count().nlargest(10).to_frame().style.bar(align = "mid", color = tyrkys)

In [ ]:
# 10 Okresů s nejvíce inzeráty
data.groupby('okres')['url_id'].count().nlargest(10).to_frame().style.bar(align = "mid", color = tyrkys)

In [ ]:
# 10 Okresů s nejvíce inzeráty bez Prahy
data[data["město"] != "Praha"].groupby( "okres")['url_id'].count().nlargest(10).to_frame().style.bar(align = "mid", color = tyrkys)

In [ ]:
# Kraje podle počtu inzerátů
data.groupby('kraj')['url_id'].count().nlargest(20).to_frame().style.bar(align = "mid", color = tyrkys)

### 2) Ceny // Ceny za metr, vše v tisících

### 2.a) Ceny podle místností

In [ ]:
# Pomocná funkce na vedle sebe grafy:) #Verze 3 - s mezerou a s popisky hned u DF
# https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

def display_side_by_side(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))


In [ ]:
# Ceny celkově
df1 = data["cena"].agg(['min', "mean", np.median,'max']).apply(lambda x: round(x)).to_frame()
df2 = data["cena_metr"].agg(['min', "mean", np.median,'max']).apply(lambda x: round(x)).to_frame()

display_side_by_side([df1, df2], ['Ceny celkem (tisíce)', 'Ceny za metr celkově (tisíce)'])

In [ ]:
#Ceny podle místností: 
df_rooms1 = data.groupby("pokoje")["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)
df_rooms2 = data.groupby("pokoje")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

df_rooms1

In [ ]:
df_rooms2


### 2.b) Ceny podle měst

In [ ]:
# Ceny celkově:
df1 = data["cena"].agg(['min', "mean", np.median,'max']).apply(lambda x: round(x)).to_frame()
df2 = data["cena_metr"].agg(['min', "mean", np.median,'max']).apply(lambda x: round(x)).to_frame()

display_side_by_side([df1, df2], ['Ceny celkem (tisíce)', 'Ceny za metr celkově (tisíce)'])

### 2.b.1) Nejdražší města

In [ ]:
# Nejdražší města
df_city1 = data.groupby(["město", "okres"])["cena"].agg(["min",  np.median, "mean","max", "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)
df_city2 = data.groupby(["město", "okres"])["cena_metr"].agg(["min",  np.median, "mean","max", "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)

df_city1
# Subset jen říká, kam chci barvu

# Upravený kod, jde řadit nLargest i podle jednoho z více prvků !!
# upravený kod, jde formátovat i některé sloupce
# 
# V1 :nechám agg( [sloupec ]) a pak musím definovat který chci v Nlargest ale mám frame a nemusím dávat "to_frame()"
# V2: nechám jen agg(sloupec) a pak nemusím definovat podle kterého chci řadit ale zas musím udělat to_frame()

In [ ]:
df_city2

### 2.b.2) Nejlevnější města

In [ ]:
# Nejlevnější města
df_city3 = data.groupby(["město", "okres"])["cena"].agg(["min",  np.median, "mean","max", "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená) 
df_city4 = data.groupby(["město", "okres"])["cena_metr"].agg(["min",  np.median, "mean","max", "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená) 

df_city3
#  sort_values( ascending = False, "median").head(15) // mi nahrazuje //   .nsmalest(15, "median"),  protože jsem chtěl aby to bylo seřazené v sestupném pořadí


In [ ]:
df_city4

### 2.c) Ceny podle okresů

### 2.c.1) Nejdražší okresy

In [ ]:
# Nejdražší okresy
df_okresy1 = data.groupby("okres")["cena"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)
df_okresy2 = data.groupby("okres")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)

df_okresy1

In [ ]:
df_okresy2

### 2.c.2) Nejlevnější okresy

In [ ]:
# Nejlevnější okresy
df_okresy3 = data.groupby("okres")["cena"].agg(['min', "mean", np.median,'max', "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená) 
df_okresy4 = data.groupby("okres")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená) 

df_okresy3

In [ ]:
df_okresy4

### 2.d) Ceny podle krajů

In [ ]:
# Ceny podle krajů
df_kraje1 = data.groupby("kraj")["cena"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = rudá)
df_kraje2 = data.groupby("kraj")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = rudá)

df_kraje1

In [ ]:
df_kraje2

### 3) Ceny podle místnosí PRAHA a MOST


### 3.a) Ceny podle místností PRAHA

In [ ]:
# Ceny podle místností PRAHA
df_rooms_PRG1 = data[data["město"] == "Praha"].groupby("pokoje")["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)
df_rooms_PRG2 = data[data["město"] == "Praha"].groupby("pokoje")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

# Postřeh: kde je větší nabídka, jsou i větší extrémy, aspoň co se  minim týče

df_rooms_PRG1

In [ ]:
df_rooms_PRG2

### 3.b) Ceny městských částí Prahy
* celkem 95 městských částí

### 3.b.1) Nejdražší pražské části

In [ ]:
# Nejdražší pražské části
df_oblast_PRG1 = data[data["město"] == "Praha"].groupby("oblast")["cena"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, columns = "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)
df_oblast_PRG2 = data[data["město"] == "Praha"].groupby("oblast")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, columns = "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)

df_oblast_PRG1

In [ ]:
df_oblast_PRG2

### 3.b.2) Nejlevnější pražské části

In [ ]:
# Nejlevnější pražské části
df_oblast_PRG3 = data[data["město"] == "Praha"].groupby("oblast")["cena"].agg(['min', "mean", np.median,'max', "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená)
df_oblast_PRG4 = data[data["město"] == "Praha"].groupby("oblast")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená)

df_oblast_PRG3

In [ ]:
df_oblast_PRG4

### 3.c) Ceny podle místností MOST

In [ ]:
# Ceny podle místností MOST

#Ten poslední jediný záznam  4+kk to trošku rozbourává ...Takže ho zkusím spíš odebrat
df_rooms_MOST1 = data[(data["město"] == "Most") & (data["pokoje"] != "4+kk")].groupby(["město", "pokoje"])["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)
df_rooms_MOST2 = data[(data["město"] == "Most") & (data["pokoje"] != "4+kk")].groupby(["město", "pokoje"])["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

df_rooms_MOST1

In [ ]:
df_rooms_MOST2

### 3.d)  PRAHA vs MOST

In [ ]:
# Cena Praha - Most podle místností

# Porovnám jen ty pokoje, které existují i v mostě, vyhnu se tím super extrémům
# Odebral jsem i 4+kk protože za Most byl jediný záznam - viz předešlý graf

df_pokoje_PRG_MOST1 = data[((data.město == "Praha") | (data.město == "Most")) & (data.pokoje != "4+kk")& (data.pokoje != "5+1") & (data.pokoje != "5+kk") & (data.pokoje != "6-a-vice") & (data.pokoje != "atypicky") & (data.pokoje != "5+1") ]\
.groupby(["pokoje","město" ])["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

df_pokoje_PRG_MOST2 = data[((data.město == "Praha") | (data.město == "Most")) & (data.pokoje != "4+kk")& (data.pokoje != "5+1") & (data.pokoje != "5+kk") & (data.pokoje != "6-a-vice") & (data.pokoje != "atypicky") & (data.pokoje != "5+1") ]\
.groupby(["pokoje","město" ])["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

df_pokoje_PRG_MOST1

In [ ]:
df_pokoje_PRG_MOST2

### Zajímavý extrém... Praha 1+kk za 10 mega -> Už ho přejmenovali na 2+kk

In [ ]:
data[(data.město == "Praha") & (data.cena >10000)  & ((data.pokoje == "1+kk")| (data.pokoje == "1+1"))]

### 4) Pivoty

### 4.a) Pivot Pražské části x Pokoje
* 95 pražských částí

In [ ]:
table = pd.pivot_table(data[data["město"] == "Praha"], values='cena', index=['oblast'],
                    columns=['pokoje'], aggfunc=np.mean).apply(lambda x: round(x))

# V1- Jen bílé nuly   # table.highlight_null(null_color='white')
# V2 - cmap = cm -> bílo-zelená    cm = sns.light_palette("green", as_cmap=True)
# V3 - cmap = "viridis"
# V4 - cmap = "magma"
# V5 - Inferno
# V6 - hot

# V5 - "magma_r"
table.style.background_gradient(cmap='magma_r')

### Nedodělaný HeatMap

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15,35)) 

sns.heatmap(table, annot=True,fmt=".0f")

### 4.b) Pivot Kraje x Pokoje

In [ ]:
table = pd.pivot_table(data, values='cena', index=['kraj'],
                    columns=['pokoje'], aggfunc=np.mean).apply(lambda x: round(x))

cm = sns.light_palette("green", as_cmap=True)
table.style.background_gradient(cmap='magma_r')



### 4.c) Pivot Kraje (bez Prahy) x Pokoje

In [ ]:
table = pd.pivot_table(data[data.město != "Praha"], values='cena', index=['kraj'],
                    columns=['pokoje'], aggfunc=np.mean).apply(lambda x: round(x))

cm = sns.light_palette("green", as_cmap=True)
table.style.background_gradient(cmap='magma_r')

In [ ]:
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "8pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "24pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '600px'),
                        ('font-size', '36pt')])
]

In [ ]:
table.style.background_gradient(cmap = "magma_r")\
    .set_properties(**{'max-width': '1200px', 'font-size': '1pt'})\
    .set_caption("Hover to magnify")\
    .set_precision(10)\
    .set_table_styles(magnify())

# Precision ovlivní délku čísla

## 5) Interaktívní widgety = DODĚLAT

In [ ]:
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

cities = widgets.Dropdown(options =  unique_sorted_values_plus_ALL(data.city_one_praha))

def dropdown_year_eventhandler(change):
    display(data[data.city_one_praha == change.new])

cities.observe(dropdown_year_eventhandler, names='value')

display(cities)

 

In [ ]:
output = widgets.Output()
display(output)

In [ ]:
from ipywidgets import interact

import jupyterlab-widgets as lab_widgets

In [ ]:
#    jupyter labextension install @jupyter-widgets/jupyterlab-manager
# Nevidím nikde svj návod, ale jel jsme jinak podle: https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916



ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique
################################

dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(data.city_one_praha))

output_year = widgets.Output()

def dropdown_year_eventhandler(change):
    output_year.clear_output()
    with output_year:
        if (change.new == ALL):
            display(data)
        else:
            display(data[data.city_one_praha == change.new])

dropdown_year.observe(dropdown_year_eventhandler, names='value')

display(dropdown_year)

        





In [ ]:
display(output_year)
